In [23]:
%config IPCompleter.greedy=True
import re
import json
import pandas as pd
from collections import defaultdict
from sklearn.feature_extraction.text import CountVectorizer
from functools import partial
from tqdm import tqdm_notebook as tqdm

Read dataset

In [4]:
cv = CountVectorizer(token_pattern=r'\w+')
analyzer = cv.build_analyzer()

In [5]:
full_q = []
full_q_freq = []
with open('yaqq_full.tsv', 'r') as inf:
    for line in inf:
        q = ' '.join(line.strip().split()[:-1])
        full_q.append(' '.join(analyzer(q)))
        full_q_freq.append(int(line.strip().split()[-1]))

### Filtering

Filters

In [84]:
# all filters

def filter_shorter_than_n(q, n=2):
    q = ' '.join(analyzer(q))
    return len(q.split()) > n

def filter_starts_with_how(q):
    q = ' '.join(analyzer(q))
    words = q.split()
    if words[0] != 'как':
        return True
    whitelist = [
        'зовут',
        'звали',
        'называется',
        'называли',
        'называют',
        'называлось',
        'назывались',
        'называлась',
        'назывался',
        'называются'
    ]
    if words[1] in whitelist:
        return True
    return False

def filter_starts_with_what(q):
    q = ' '.join(analyzer(q))
    words = q.split()
    if words[0] != 'что':
        return True
    blacklist = [
        'будет',
        'нужно',
        'делать',
        'можно',
        'лучше'
    ]
    if words[1] in blacklist:
        return False
    return True

def filter_stop_words(q):
    q = ' '.join(analyzer(q))
    stop_words = set([
        'купить',
        'почему',
        'онлайн',
        'сегодня',
        'лучше',
        'зачем',
        'снится',
        'приготовить',
        'беременности',
        'видео',
        'документы',
        'тест', 
        'подарить', 
        'снятся', 
        'калорий',
        'беременным', 
        'бесплатно', 
        'нужен',
        'нужна',
        'нужно',
        'нужны',
        'отдохнуть', 
        'сниться', 
        'нельзя', 
        'заниматься', 
        'беременность', 
        'отдыхать', 
        'выбрать',
        'забеременеть', 
        'месячных', 
        'сексом', 
        'похудеть', 
        'контакте', 
        'загранпаспорт',
        'сайт',
        'может',
        'фото'
    ])
    words = q.split()
    for word in words:
        if word in stop_words:
            return False
    return True

def filter_stop_substr(q):
    q = ' '.join(analyzer(q))
    stop_substr = [
        '100 к 1',
        'сто к одному',
        'можно ли',
        'где можно'
    ]
    for sb in stop_substr:
        if sb in q:
            return False
    return True

def filter_starts_with_stop_words(q):
    q = ' '.join(analyzer(q))
    stop_words = [
        'игра',
        'игры',
        'с чем',
        'чем ',
        'сколько стоит',
        'где отметить',
        'где отпраздновать',
        'что подарить',
        'сколько времени в '
    ]
    for sw in stop_words:
        if q.startswith(sw):
            return False
    return True

def filter_must_start_with_question_word(q):
    ws = ' '.join(analyzer(q))
    q_words = [
        'кто ',
        'где ',
        'когда ',
        'какой ',
        'каком '
    ]
    for qw in q_words:
        if qw in ws:
            return True
    return False
    

def filter_unknown_words(q):
    ws = analyzer(q)
    for w in ws:
        if w not in russ_vocab:
            return False
    return True

Run filters

In [7]:
def apply_filters(q, filters):
    new_q = q
    for f in filters:
        new_q = list(filter(f, new_q))
    return new_q

def print_q(qs):
    for q in qs:
        print(q)

In [85]:
all_filters = [
    filter_shorter_than_n,
    filter_starts_with_how,
    filter_starts_with_what,
    filter_stop_words,
    filter_stop_substr,
    filter_starts_with_stop_words
]

filtered_q = apply_filters(
    q=full_q,
    filters=all_filters
)

In [86]:
print(len(filtered_q))
cnt = 0
for q in filtered_q:
    if 'нужн' in q:
        cnt += 1
        print(q)
print(cnt)

353130
сколько нежных слов я не сказал сколько их ненужных обронил
сколько серий в сериале остров ненужных людей
куда отдать ненужные вещи
куда отдать ненужную одежду
где продать ненужные вещи
где снимался фильм остров ненужных людей
где снимали остров ненужных людей
сколько серий остров ненужных людей
где снимали фильм остров ненужных людей
сколько серий в фильме остров ненужных людей
что полезного и нужного вы могли бы предложить компании
будет ли продолжение сериала остров ненужных людей
сколько серий в острове ненужных людей
потому что потому что всех нужнее и дороже
куда сдать ненужную одежду
куда можно отдать ненужную одежду
программа которая удаляет ненужные файлы
куда деть ненужную одежду
куда сдать ненужные вещи
где снимался остров ненужных людей
куда деть ненужные книги
сколько серий в остров ненужных людей
где снимался сериал остров ненужных людей
на каком острове снимался фильм остров ненужных людей
где снимали сериал остров ненужных людей
куда деть ненужные вещи
куда можно

### Properties search

Search methods

In [9]:
def finder_substr(q, prop):
    if re.search(prop, q) is not None:
        return True
    return False

Run search

In [10]:
def apply_properties_finder(q, finder, prop):
    f = partial(finder, prop=prop)
    return list(filter(f, q))

In [11]:
def find_property(aliases):
    qs = []
    for alias in aliases:
        cur_qs = apply_properties_finder(filtered_q, finder_substr, alias)
        qs.extend(cur_qs)
    return qs

Load aliases

In [91]:
with open('properties_forms.txt', 'r') as inf:
    res = {}
    pid = None
    aliases = []
    for line in inf:
        line = line.strip('\n')
        if line.startswith('Count') or \
           line.startswith('Description') or \
           line.startswith('Aliases') or \
           line.startswith('Question') or \
           line.startswith('Label') or \
           line.startswith('# ') or \
           not line:
                continue
        elif line.startswith('####'):
            res[pid] = aliases
            pid = None
            aliases = []
        elif line.startswith('PID'):
            pid = line.replace('PID: ', '')
        else:
            aliases.append(line)

with open('properties_aliases.json', 'w') as ouf:
    json.dump(res, ouf, ensure_ascii=False, indent=4)

In [92]:
with open('properties_aliases.json', 'r') as inf:
    prop_aliases = json.load(inf)

Load PIDs

In [89]:
with open('pids.json', 'r') as inf:
    pids = json.load(inf)

Search

In [65]:
find_property(prop_aliases['P577'])

['когда впервые и кем уральские горы были названы своим современным именем',
 'кто и когда впервые использовал термин культурология',
 'когда впервые были анонсированы usb накопители verbatim micro на российском рынке',
 'когда впервые стали использовать наволочку',
 'когда впервые открылся рождественский базар винер адвентцаубер',
 'когда впервые перевели время в ссср',
 'кем и когда впервые получена медь',
 'когда впервые появился термин социология',
 'когда впервые и кем уральские горы были названы своим современным именем',
 'когда впервые перевели часы']

In [66]:
prop_aliases['P1559']

['имя при рождении', 'на родном языке', 'оригинал имени', 'оригинальное имя']

In [93]:
retrieved_table = []
with open('retrieved.txt', 'w') as ouf:
    pass
with open('retrieved.txt', 'a') as ouf:
    for pid in tqdm(pids[:100]):
        print(pid[0], file=ouf)
        print(pid[1], file=ouf)
        print(prop_aliases[pid[0]], file=ouf)
        for q in find_property(prop_aliases[pid[0]]):
            print(q, file=ouf)
            row = {}
            row['pid'] = pid[0]
            row['label'] = pid[1]
            row['q'] = q
            row['good'] = ''
            retrieved_table.append(row)
        print(file=ouf)

retrieved_df = pd.DataFrame(retrieved_table)[['pid', 'label', 'q', 'good']]

In [94]:
retrieved_df

,pid,label,q,good
0,P577,дата публикации,когда впервые и кем уральские горы были назван...,
1,P577,дата публикации,кто и когда впервые использовал термин культур...,
2,P577,дата публикации,когда впервые были анонсированы usb накопители...,
3,P577,дата публикации,когда впервые стали использовать наволочку,
4,P577,дата публикации,когда впервые открылся рождественский базар ви...,
5,P577,дата публикации,когда впервые перевели время в ссср,
6,P577,дата публикации,кем и когда впервые получена медь,
7,P577,дата публикации,когда впервые появился термин социология,
8,P577,дата публикации,когда впервые и кем уральские горы были назван...,
9,P577,дата публикации,когда впервые перевели часы,


In [83]:
for q in retrieved_df[retrieved_df['pid'] == 'P50']['q']:
    print(q)

кто автор этой картины на которой изображен мчащийся поезд
кто автор картины на которой изображен мчащийся поезд
кто автор картины савинский монастырь под москвой
кто автор рассказа первая охота
кто автор картины мчащийся поезд
кто автор винни пуха
кто автор буратино
кто автор слово о полку игореве
кто автор гимна россии
кто автор библии
кто автор картины на которой изображён мчащийся поезд
кто автор самого известного конструктора игр game maker
кто автор слова о полку игореве
кто автор стихотворения
кто автор теории относительности
кто автор древней географии состоящей из 17 книг
кто автор сказки золушка
кто автор музыки гимна россии
кто автор сказки айболит
кто автор повести временных лет
кто автор сказки кот в сапогах
кто автор теории относительности
кто автор сказки дюймовочка
от чего же так не возьму я в толк кто автор
кто автор дядя степа
кто автор статуи зевса олимпийского
кто автор маугли
отчего же так не возьму я в толк кто автор
кто автор картины голубые ели
кто автор вера и 

In [96]:
retrieved_df['pid'].value_counts()

P276     3911
P585     2181
P921     1568
P2047    1345
P50      1255
P527     1249
P2067    1158
P735      823
P607      773
P2048     721
P131      702
P156      610
P17       584
P119      573
P582      551
P462      532
P166      415
P569      402
P27       391
P1082     387
P571      378
P127      373
P175      348
P1619     347
P106      307
P576      300
P282      297
P463      297
P40       295
P101      267
         ... 
P57        54
P2046      46
P108       43
P3373      42
P102       27
P1344      27
P21        26
P1539      17
P136       16
P179       14
P1810      13
P20        12
P162       12
P577       10
P123       10
P969        9
P495        9
P264        9
P2049       5
P291        5
P159        4
P512        4
P937        4
P58         3
P274        3
P478        3
P413        3
P137        2
P3744       1
P1532       1
Name: pid, Length: 86, dtype: int64

In [99]:
retrieved_df_unique = retrieved_df.drop_duplicates(subset=['q'], keep='first')

In [101]:
retrieved_df_unique.to_csv('dirty_questions_from_top100_properties.csv', index=False)